In [27]:
%load_ext autoreload
%autoreload 2

from biolearn.data_library import DataLibrary
import pickle
from tqdm import tqdm
import textwrap
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu, wilcoxon
from statsmodels.stats.multitest import multipletests
from os.path import basename, splitext, exists
from glob import glob

from computage.utils.data_utils import test_dataset
from computage.configs.datasets_bench_config import *
from computage.benchmarking.benchmarking import EpiClocksBenchmarking

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# test call of dataset

In [166]:
gse = 'GSE52588'
data_source = DataLibrary().get(gse)

print(textwrap.fill(data_source.summary, 70))
data=data_source.load()

Down syndrome is characterized by a wide spectrum of clinical signs,
which include cognitive and endocrine disorders and haematological
abnormalities. Although it is well established that the causative
defect of Down syndrome is the trisomy of chromosome 21, the molecular
bases of Down syndrome phenotype are still largely unknown. We used
the Infinium HumanMethylation450 BeadChip to investigate DNA
methylation patterns in whole blood from 29 subjects affected by Down
syndrome (DS), using their healthy relatives as controls (mothers  and
unaffected siblings). This family-based model allowed us to monitor
possible confounding effects on DNA methylation patterns deriving from
genetic and environmental (lifestyle) factors. The identified
epigenetic signature of Down syndrome includes differentially
methylated regions that, although enriched on chromosome 21, interest
most of the other chromosomes and can be functionally linked to the
developmental and haematological defects characteristic 

In [167]:
data_obj = {}
data_obj['data'] = data.dnam
data_obj['meta'] = data.metadata

In [168]:
pickle.dump(data_obj, open(f'data/{gse}.pkl', 'wb'))

In [169]:
#now load it
data_obj = pickle.load(open(f'data/{gse}.pkl', 'rb'))

In [171]:
data_obj['meta']['disease_state']#.value_counts()

id
GSM1272122    Down syndrome
GSM1272123    Down syndrome
GSM1272124    Down syndrome
GSM1272125    Down syndrome
GSM1272126    Down syndrome
                  ...      
GSM1272204          healthy
GSM1272205          healthy
GSM1272206          healthy
GSM1272207          healthy
GSM1272208          healthy
Name: disease_state, Length: 87, dtype: object

# test dataset downloading

In [1]:
import requests
from urllib.parse import urlencode
import os

base_url = 'https://cloud-api.yandex.net/v1/disk/public/resources/download?'
public_key = 'https://disk.yandex.ru/i/6EKzmWgHWfQhhA'  # link to file

# Получаем загрузочную ссылку
final_url = base_url + urlencode(dict(public_key=public_key))
response = requests.get(final_url)
download_url = response.json()['href']
download_url

'https://downloader.disk.yandex.ru/disk/c3497b4318bdb37d31e77c872507055652550449e3a53e42ad020f4553360079/6602bc7e/o-xAw257UInz2E5KCNNYxzrwZIFIEChaHjZ5c8XuVTkQQP5Vu3LX_PfcrbWl_WXMgzctlrPIR5lis-RHW_1Y4w%3D%3D?uid=0&filename=jess.jpg&disposition=attachment&hash=PIh7RkISs8vpRICogGJFgYXo7GoHNggOK3PpICkoZx%2BkzyvR9xvEdnuySuRZLXysq/J6bpmRyOJonT3VoXnDag%3D%3D%3A&limit=0&content_type=image%2Fjpeg&owner_uid=614251127&fsize=3052875&hid=53e1d3b50896254829e1f189ba2f4197&media_type=image&tknv=v2'

In [ ]:
# Загружаем файл и сохраняем его
download_response = requests.get(download_url)
with open('jess.jpg', 'wb') as f:   # Здесь укажите нужный путь к файлу
    f.write(download_response.content)

In [ ]:
download_url = 'https://disk.yandex.ru/d/UKAe_mXsv2TfVg'
out_file_name = 'jess.txt'

with open(os.path.join(out_file_name), 'wb') as out_stream:
    req = requests.get(download_url, stream=True)
    for chunk in req.iter_content(1024):  # Куски по 1 КБ
        out_stream.write(chunk)

# Benchmarking of published clocks

In [31]:
models_config = {
    "in_library":{
        'Horvathv1':{},
        'Hannum':{},
        'Lin':{},
        'PhenoAge':{},
        'YingCausAge':{},
        'YingDamAge':{},
        'YingAdaptAge':{},
        'Horvathv2':{},
        'PEDBE':{},
        'HRSInCHPhenoAge':{},
    },
    #each model should have `path` in its dict values (see example)
    #each model should be stored in pickle (.pkl) format
    "new_models":{
        #'my_new_model_name': {'path':/path/to/model.pkl}
        
    }
}

datasets_config_short = dict(list(datasets_config.items())[:3])


In [29]:
#fill config with new models
pattern = '/tank/projects/computage/checkpoints/*fs3*'
for p in glob(pattern):
    name = splitext(basename(p))[0]
    models_config['new_models'][name] = {'path':p}

In [33]:
data, meta = pd.read_pickle('/tank/projects/computage/benchmarking/GSE56606.pkl.gz', compression='gzip').values()

In [32]:
bench = EpiClocksBenchmarking(
    models_config=models_config,
    datasets_config=datasets_config_main,
    tissue_types='Blood',
    experiment_prefix='prefinal0',
    correction_threshold=0.05,
    save_results=True,
    output_folder='./tmp_bench_results',
    verbose=1
)
bench.run()

10 models will be tested on 60 datasets.


Datasets:   2%|▏         | 1/60 [00:58<57:45, 58.74s/it]

GSE56046:AS - AA2 testing 776 disease versus 339 healthy samples


Datasets:   3%|▎         | 2/60 [01:07<28:19, 29.30s/it]

GSE56581:AS - AA2 testing 127 disease versus 66 healthy samples


...

Datasets:  60%|██████    | 36/60 [05:55<01:44,  4.34s/it]

GSE49909:OBS - AA2 testing 31 disease versus 40 healthy samples
GSE49909:ASD - 0 disease and 40 healthy samples found - AA2 test is impossible. Skip!


Datasets:  60%|██████    | 36/60 [05:56<03:57,  9.90s/it]

GSE56606:T1D - AA2 testing 32 disease versus 68 healthy samples


TypeError: unsupported operand type(s) for -: 'float' and 'str'

In [12]:
bench.corrected_results_bool.loc[s.index]

,IBD:GSE32148:AAP,AS:GSE56581:AAP,IBD:GSE87640:AAP,HIV:GSE67751:AAP,RA:GSE131989:AAP,RA:GSE134429:AAP,OBS:GSE49909:AAP,OP:GSE99624:AAP,WS:GSE131752:AAP,HGPS:GSE182991:AAP,CGL:GSE214297:AAP
pc_kdm_scale_9_bench_fs3_ultrav1,True,False,True,True,False,False,False,False,True,False,True
pc_kdm_scale_8_bench_fs3_ultrav1,True,False,True,True,False,False,False,False,True,False,True
pc_lin_9_bench_fs3_ultrav1,True,False,True,True,False,False,False,False,True,False,True
pls1_4_bench_fs3_ultrav1,False,False,True,True,False,False,True,False,True,False,False
PhenoAge,False,False,True,True,False,False,True,False,True,False,False
HRSInCHPhenoAge,False,False,True,True,False,True,False,False,True,False,False
pc_lin_8_bench_fs3_ultrav1,False,False,False,True,False,False,False,False,True,False,True
pls1_8_bench_fs3_ultrav1,False,False,False,True,True,False,False,False,True,False,False
pls1_kdm_5_bench_fs3_ultrav1,False,False,True,True,False,False,False,False,True,False,False
pc_kdm_scale_5_bench_fs3_ultrav1,False,False,False,True,False,False,False,False,True,False,True


In [10]:
s = bench.corrected_results_bool.sum(1).sort_values(ascending=False)#.index
s

pc_kdm_scale_9_bench_fs3_ultrav1    5
pc_kdm_scale_8_bench_fs3_ultrav1    5
pc_lin_9_bench_fs3_ultrav1          5
pls1_4_bench_fs3_ultrav1            4
PhenoAge                            4
HRSInCHPhenoAge                     4
pc_lin_8_bench_fs3_ultrav1          3
pls1_8_bench_fs3_ultrav1            3
pls1_kdm_5_bench_fs3_ultrav1        3
pc_kdm_scale_5_bench_fs3_ultrav1    3
pls1_5_bench_fs3_ultrav1            3
pls1_3_bench_fs3_ultrav1            3
YingCausAge                         2
Horvathv1                           2
pls1_kdm_6_bench_fs3_ultrav1        2
pls1_6_bench_fs3_ultrav1            2
pls1_9_bench_fs3_ultrav1            2
pc_kdm_scale_6_bench_fs3_ultrav1    2
YingAdaptAge                        2
Lin                                 2
Horvathv2                           2
PEDBE                               2
pls1_kdm_4_bench_fs3_ultrav1        2
pc_lin_5_bench_fs3_ultrav1          2
pls1_kdm_8_bench_fs3_ultrav1        2
pls1_kdm_2_bench_fs3_ultrav1        2
pls1_kdm_9_b

In [5]:
# dnam, meta = pd.read_pickle(f'{PREFIX}GSE87648.pkl.gz', compression='gzip').values()

In [204]:
listmodels = glob('/tank/projects/computage/checkpoints/pls*pheno*ultra*')
listmodels, len(listmodels)

(['/tank/projects/computage/checkpoints/pls1_11_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_7_kdm_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_6_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_1_kdm_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_12_kdm_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_6_kdm_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_7_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_13_kdm_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_10_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_1_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_4_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_9_kdm_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_13_pheno_ultrav1.pkl',
  '/tank/projects/computage/checkpoints/pls1_3_kdm_pheno_ultrav1.pkl',
  '/tank/projects/computage/c

# legacy

In [2]:
# from biolearn.data_library import GeoData
# from biolearn.model_gallery import ModelGallery

# published = False

# gallery = ModelGallery()

# bench_results_AAP = pd.DataFrame()
# bench_results_AA0 = pd.DataFrame()
# datasets_predictions = {}
# for gse, conf in datasets_config.items():
#     #import data
#     path, cond, test = conf.values()
#     dnam, meta = pd.read_pickle(path, compression='gzip').values()
#     data = GeoData(meta, dnam.T)

#     ###Predict datasets and gather predictions
#     #Note that by default clocks will impute missing data.
#     #To change this behavior set the imputation= parameter when getting the clock #???

#     predictions = {}
#     if published:
#         #published clocks prediction
#         keys = list(gallery.model_definitions.keys())
#         for k in tqdm(keys):
#             try:
#                 if gallery.model_definitions[k]['output']=='Age (Years)':
#                     results = gallery.get(k, imputation_method='none').predict(data)
#                     predictions[k] = results['Predicted']
#             except:
#                 print('Oops!')
#                 continue
#     else:
#         #de novo clocks prediction
#         for path in tqdm(listmodels):
#             k = splitext(basename(path))[0]
#             model = pickle.load(open(path, 'rb'))
#             try:
#                 dnam_ = dnam.reindex(columns = list(model.pls.feature_names_in_)).copy()
#             except:
#                 dnam_ = dnam.reindex(columns = list(model.feature_names_in_)).copy()
#                 dnam_ = dnam_.fillna(0.)
            
#             preds_ = model.predict(dnam_)
#             if type(preds_) == np.ndarray:
#                 predictions[k] = pd.Series(preds_, index=dnam.index)
#             else:
#                 predictions[k] = pd.Series(preds_.values, index=dnam.index)
        
#     pred = pd.DataFrame(predictions)
#     datasets_predictions[gse] = pred.copy()
#     #meta filtering
#     no_age_na_indices = meta[~meta['Age'].isna()].index
#     meta = meta.loc[no_age_na_indices]
#     if test == 'AAP':
#         #calculating mann-whitney test for difference in age acceleration between disease and healthy cohorts
#         disease_idx = meta.index[meta['Condition'] == cond]
#         healthy_idx = meta.index[meta['Condition'] == 'HC']
#         print(f'{cond}:{gse} - AAP testing {len(disease_idx)} disease versus {len(healthy_idx)} healthy samples')
#         pvals = {}
#         for col in pred.columns:
#             disease_true = meta.loc[disease_idx, 'Age'].values
#             healthy_true = meta.loc[healthy_idx, 'Age'].values
#             disease_pred = pred.loc[disease_idx, col].values
#             healthy_pred = pred.loc[healthy_idx, col].values
#             disease_delta = disease_pred - disease_true
#             healthy_delta = healthy_pred - healthy_true
#             stat, pval = mannwhitneyu(disease_delta, healthy_delta, alternative='greater')
#             pvals[col] = pval
#         bench_results_AAP[f'{cond}:{gse}:AAP'] = pd.Series(pvals)
#     elif test == 'AA0':
#         #calculating wilcoxon test for positive age (>0) acceleration in disease cohort
#         disease_idx = meta.index[meta['Condition'] == cond]
#         print(f'{cond}:{gse} - AA0 testing {len(disease_idx)} disease samples')
#         pvals = {}
#         for col in pred.columns:
#             disease_true = meta.loc[disease_idx, 'Age'].values
#             disease_pred = pred.loc[disease_idx, col].values
#             disease_delta = disease_pred - disease_true
#             stat, pval = wilcoxon(disease_delta, alternative='greater')
#             pvals[col] = pval
#         bench_results_AA0[f'{cond}:{gse}:AA0'] = pd.Series(pvals)
#     else:
#         NotImplementedError("Only two tests are currently available: ['AAP', 'AA0'].")
    

In [167]:
#dnam, meta = pd.read_pickle(datasets_config['GSE53840']['path'], compression='gzip').values()

In [3]:
# def correction(x):
#     return multipletests(x, method='fdr_bh')[1]

# bench_results = pd.concat([bench_results_AAP, bench_results_AA0], axis=1).dropna(axis=0)
# corrected_results_AAP = bench_results_AAP.T.apply(correction, axis=0).T < 0.05
# corrected_results_AA0 = bench_results_AA0.T.apply(correction, axis=0).T < 0.05
# corrected_results = pd.concat([corrected_results_AAP, corrected_results_AA0], axis=1)

# print(corrected_results.shape)
# corrected_results.sum(axis=1).sort_values(ascending=False)

# bench_results.to_csv('/tank/projects/computage/results/bench_results_pls_pheno_ultra.csv')
# corrected_results.to_csv('/tank/projects/computage/results/bench_results_pls_pheno_ultra.csv')

In [15]:
pd.read_csv('/tank/projects/computage/results/bench_results_pre_noimputation_corrected.csv', index_col=0)#.sum(axis=1).sort_values(ascending=False)

,DS:GSE52588,Rheumatoid arthritis:GSE42861,AD:GSE59685,AD:GSE80970,Overweight:GSE73103,IBD:GSE87640,IBD:GSE87648,HT:GSE157131
Horvathv1,True,False,False,False,False,False,False,False
Hannum,True,True,False,False,False,True,True,False
Lin,True,False,False,False,False,True,False,False
PhenoAge,True,True,False,False,False,True,True,True
YingCausAge,True,False,False,False,False,False,False,False
YingDamAge,False,False,False,False,False,False,True,False
YingAdaptAge,True,False,False,False,False,False,False,False
Horvathv2,True,False,False,False,False,False,False,False
PEDBE,True,False,False,False,False,False,False,False
HRSInCHPhenoAge,True,True,False,False,True,True,True,True


In [13]:
pd.read_csv('/tank/projects/computage/results/bench_results_ultrav1_train_pheno_corrected.csv', index_col=0)#.sum(axis=1).sort_values(ascending=False)

,DS:GSE52588,Rheumatoid arthritis:GSE42861,AD:GSE59685,AD:GSE80970,Overweight:GSE73103,IBD:GSE87640,IBD:GSE87648,HT:GSE157131
pls1_11_pheno_ultrav1,True,False,False,False,False,False,False,False
pls1_7_kdm_pheno_ultrav1,True,False,False,False,False,False,True,False
pls1_6_pheno_ultrav1,True,False,False,False,False,False,False,False
pls1_1_kdm_pheno_ultrav1,True,True,False,False,False,False,False,False
pls1_12_kdm_pheno_ultrav1,True,False,False,False,False,False,True,False
pls1_6_kdm_pheno_ultrav1,True,False,False,False,False,False,True,False
pls1_7_pheno_ultrav1,True,False,False,False,False,False,False,False
pls1_13_kdm_pheno_ultrav1,True,False,False,False,False,False,True,False
pls1_10_pheno_ultrav1,True,False,False,False,False,False,False,False
pls1_1_pheno_ultrav1,True,True,False,False,False,False,False,False


In [23]:
pd.read_csv('/tank/projects/computage/results/bench_results_kdm_corrected.csv', index_col=0)

,DS:GSE52588,Rheumatoid arthritis:GSE42861,AD:GSE59685,AD:GSE80970,Overweight:GSE73103,IBD:GSE87640,IBD:GSE87648,HT:GSE157131
kdm_rse_all_20k,True,True,False,False,False,False,True,False
kdm_rse_forward_20k,False,False,False,False,True,True,False,False


In [84]:
bench_results_logical = bench_results < 0.05
bench_results_logical

In [49]:
#look up at all models
pd.DataFrame(gallery.model_definitions).T

,year,species,tissue,source,output,model
Horvathv1,2013,Human,Multi-tissue,https://genomebiology.biomedcentral.com/articl...,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'Ho..."
Hannum,2013,Human,Blood,https://www.sciencedirect.com/science/article/...,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'Ha..."
Lin,2016,Human,Blood,https://www.aging-us.com/article/100908/text,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'Li..."
PhenoAge,2018,Human,Blood,https://www.aging-us.com/article/101414/text,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'Ph..."
YingCausAge,2022,Human,Blood,https://www.biorxiv.org/content/10.1101/2022.1...,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'Yi..."
YingDamAge,2022,Human,Blood,https://www.biorxiv.org/content/10.1101/2022.1...,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'Yi..."
YingAdaptAge,2022,Human,Blood,https://www.biorxiv.org/content/10.1101/2022.1...,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'Yi..."
Horvathv2,2018,Human,Skin + blood,https://www.aging-us.com/article/101508/text,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'Ho..."
PEDBE,2019,Human,Buccal,https://www.pnas.org/doi/10.1073/pnas.1820843116,Age (Years),"{'type': 'LinearMethylationModel', 'file': 'PE..."
Zhang_10,2019,Human,Blood,https://www.nature.com/articles/ncomms14617,Mortality Risk,"{'type': 'LinearMethylationModel', 'file': 'Zh..."
